In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



In [2]:
df_train = pd.read_csv("./train.csv/train.csv",nrows=50000)
df_train.head()
df_train.info()
df_train.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   key                50000 non-null  object 
 1   fare_amount        50000 non-null  float64
 2   pickup_datetime    50000 non-null  object 
 3   pickup_longitude   50000 non-null  float64
 4   pickup_latitude    50000 non-null  float64
 5   dropoff_longitude  50000 non-null  float64
 6   dropoff_latitude   50000 non-null  float64
 7   passenger_count    50000 non-null  int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 3.1+ MB


(50000, 8)

In [3]:
df_train.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,11.364171,-72.509756,39.933759,-72.504616,39.926251,1.667840
std,9.685557,10.393860,6.224857,10.407570,6.014737,1.289195
min,-5.000000,-75.423848,-74.006893,-84.654241,-74.006377,0.000000
25%,6.000000,-73.992062,40.734880,-73.991152,40.734372,1.000000
50%,8.500000,-73.981840,40.752678,-73.980082,40.753372,1.000000
75%,12.500000,-73.967148,40.767360,-73.963584,40.768167,2.000000
max,200.000000,40.783472,401.083332,40.851027,43.415190,6.000000


In [4]:
df_train.isnull().sum()

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [5]:
df_train.dropna(inplace=True, axis=0)
df_train.isnull().sum()

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [6]:
print(len(df_train[df_train["pickup_longitude"]==0]))

955


In [7]:
df_train = df_train.drop(df_train[df_train["pickup_longitude"]==0].index)

In [8]:
df_train['pickup_datetime'] = pd.to_datetime(df_train['pickup_datetime'])

In [9]:
df_train.head()


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21+00:00,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16+00:00,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42+00:00,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1


In [10]:
df_train.loc[:,'pickup_date'] = df_train['pickup_datetime'].dt.day

In [11]:
df_train.loc[:, 'pickup_month'] = df_train['pickup_datetime'].dt.month
df_train.loc[:, 'pickup_year'] = df_train['pickup_datetime'].dt.year

In [12]:
df_train.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_date,pickup_month,pickup_year
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21+00:00,-73.844311,40.721319,-73.841610,40.712278,1,15,6,2009
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16+00:00,-74.016048,40.711303,-73.979268,40.782004,1,5,1,2010
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2,18,8,2011
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42+00:00,-73.987130,40.733143,-73.991567,40.758092,1,21,4,2012
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1,9,3,2010


In [13]:
# Haversine distance 
from math import cos, asin, sqrt, pi

def distance(lat1, lon1, lat2, lon2):
    r = 6371
    p = pi / 180

    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 2 * r * asin(sqrt(a))

df_train["Haversine_distance"] = df_train.apply(lambda row: distance(row['pickup_latitude'],row['pickup_longitude'],row['dropoff_latitude'],row['dropoff_longitude']), axis=1)


In [14]:
len(df_train)

49045

In [15]:
print(df_train['Haversine_distance'].median(),
df_train['Haversine_distance'].mean(),
df_train['Haversine_distance'].mode())

2.162233839130312 11.76349130528574 0    0.0
Name: Haversine_distance, dtype: float64


In [16]:
# import osmnx as ox
# import networkx as nx

In [17]:
# This is to save the NY map as a graph I think
# city = [
#     {
#         "city": "New York",
#         "state": "New York",
#         "country": "USA",
#     }
# ]
# NY_graph = ox.graph_from_place(city, network_type='drive')

In [18]:
# NY_graph2 = ox.graph_from_place('New York, USA', network_type='drive_service')

In [19]:
# ox.save_graphml(NY_graph, 'NY1.graphml')
# ox.save_graphml(NY_graph2, 'NY2.graphml')

In [ ]:
# import os
# 
# # print(os.path.isfile("./NY1.graphml") and  os.path.isfile("./NY2.graphml"))
# 
# if not (os.path.isfile("./NY1.graphml") and os.path.isfile("./NY2.graphml")):
#     city = [
#         {
#             "city": "New York",
#             "state": "New York",
#             "country": "USA",
#         }
#     ]
#     NY_graph = ox.graph_from_place(city, network_type='drive')
#     ox.save_graphml(NY_graph, 'NY1.graphml')
# 
#     NY_graph2 = ox.graph_from_place('New York, USA', network_type='drive_service')
#     ox.save_graphml(NY_graph2, 'NY2.graphml')
# 
#     print("Graphs GENERATED successfully")
# else:
#     NY_graph = ox.load_graphml('NY1.graphml')
#     NY_graph2 = ox.load_graphml('NY2.graphml')
#     print("Graphs LOADED successfully")

In [21]:
# NY_graph
# NY_graph2

In [22]:
# start_graph_node = ox.utils.get_nearest_node(NY_graph,(40.767937, -73.982155))
# end_graph_node = ox.utils.get_nearest_node(NY_graph,(40.765602, -73.964630))

# get_nearest_node is deprecated i think

In [23]:
# start_graph_node = ox.nearest_nodes(NY_graph,-73.982155,40.767937 ,return_dist=True)
# end_graph_node = ox.nearest_nodes(NY_graph,-73.982155,40.765602,return_dist=True)

# Syntax: nearest_nodes(graph_obj, longitude value, latitude value, if we want the distance or not)

In [25]:
# import folium

# m = ox.plot_graph_folium(NY_graph)
# m.save('NY1.html')

# m2 = ox.plot_graph_folium(NY_graph2)
# m2.save('NY2.html')

# folium is bad use GeoPanda or smthng
# import geopandas as gpd

In [26]:
# THIS METHOD IS IRRELEVANT TAKES TOO MUCH TIME
# from functools import lru_cache
# 
# noPathCounter = 0
# @lru_cache(maxsize=10000)
# def get_road_dist(graph_obj, long1, lat1, long2, lat2):
#     global noPathCounter
#     start_graph_node,start_node_dist = ox.nearest_nodes(graph_obj,long1,lat1,return_dist=True)
#     end_graph_node,end_node_dist = ox.nearest_nodes(graph_obj,long2,lat2,return_dist=True)
# 
#     # distance = ox.shortest_path(graph_obj,start_graph_node,end_graph_node,weight="length", )
#     try:
#         shortest_distance = nx.shortest_path_length(graph_obj,start_graph_node,end_graph_node)
#         # print("shortest path found for node:",start_graph_node,end_graph_node)
#         return shortest_distance +start_node_dist + end_node_dist
#     except nx.NetworkXNoPath as noPathError:
#         shortest_distance = None
#         noPathCounter +=1
#         print(noPathError)
#         return shortest_distance
# 
# df_train["osmnx_distance"] = df_train.apply(lambda row: get_road_dist(NY_graph,row["pickup_longitude"],row["pickup_latitude"],row["dropoff_longitude"],row["dropoff_latitude"]), axis=1)
# df_train.head()

In [ ]:
# import requests

# def get_road_dist(row):
#     start_lon, start_lat = row["pickup_longitude"], row["pickup_latitude"]
#     end_lon, end_lat = row["dropoff_longitude"], row["dropoff_latitude"]
#     
#     url = "http://router.project-osrm.org/route/v1/driving/"
#     coords = f"{start_lon},{start_lat};{end_lon},{end_lat}"
#     params = {
#         "geometries": "polyline",
#         "overview": "full",
#         "steps": "true"
#     }
#     
#     try:
#         response = requests.get(url + coords, params=params)
#         response.raise_for_status()
#         data = response.json()
#         
#         if data["code"] == "Ok" and data["routes"]:
#             return data["routes"][0]["distance"]
#         else:
#             print(f"Error: {data['message']}")
#             return None
#     except requests.exceptions.RequestException as e:
#         print(f"Error: {e}")
#         return None
# 
# 
# df_train["osrm_dist"] = df_train.apply(get_road_dist, axis=1)

In [ ]:
# import requests
# from itertools import islice
# from functools import lru_cache
# 
# BATCH_SIZE = 200 
# CACHE_SIZE = 10000 
# 
# @lru_cache(maxsize=CACHE_SIZE)
# def get_road_dist_cached(start_lon, start_lat, end_lon, end_lat):
#     url = "http://router.project-osrm.org/route/v1/driving/"
#     coords = f"{start_lon},{start_lat};{end_lon},{end_lat}"
#     params = {
#         "geometries": "polyline",
#         "overview": "full",
#         "steps": "true"
#     }
# 
#     try:
#         response = requests.get(url + coords, params=params)
#         response.raise_for_status()
#         data = response.json()
# 
#         if data["code"] == "Ok" and data["routes"]:
#             return data["routes"][0]["distance"]
#         else:
#             print(f"Error: {data['message']}")
#             return None
#     except requests.exceptions.RequestException as e:
#         print(f"Error: {e}")
#         return None
# 
# def batch(iterable, batch_size):
#     iterator = iter(iterable)
#     while True:
#         batch = list(islice(iterator, batch_size))
#         if not batch:
#             break
#         yield batch
# 
# def get_road_dist_batch(coords):
#     distances = []
#     for start_lon, start_lat, end_lon, end_lat in coords:
#         distance = get_road_dist_cached(start_lon, start_lat, end_lon, end_lat)
#         distances.append(distance)
# 
#     return distances
# 
# coords = list(df_train[["pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"]].itertuples(index=False, name=None))
# df_train["osrm_dist"] = get_road_dist_batch(coords)

In [ ]:
# import requests
# from itertools import islice
# from functools import lru_cache
# import time
# 
# BATCH_SIZE = 100  
# CACHE_SIZE = 10000
# SLEEP_TIME = 1  # Sleep time between batches to avoid rate limiting
# 
# @lru_cache(maxsize=CACHE_SIZE)
# def get_road_dist_cached(start_lon, start_lat, end_lon, end_lat):
#     url = "http://router.project-osrm.org/route/v1/driving/"
#     coords = f"{start_lon},{start_lat};{end_lon},{end_lat}"
#     params = {
#         "geometries": "polyline",
#         "overview": "full",
#         "steps": "true"
#     }
# 
#     try:
#         response = requests.get(url + coords, params=params)
#         response.raise_for_status()
#         data = response.json()
# 
#         if data["code"] == "Ok" and data["routes"]:
#             distance = data["routes"][0]["distance"]
#         else:
#             print(f"Error: {data['message']}")
#             distance = None
# 
#         return distance
#     except requests.exceptions.RequestException as e:
#         print(f"Error: {e}")
#         return None
# 
# def batch(iterable, batch_size):
#     iterator = iter(iterable)
#     while True:
#         batch = list(islice(iterator, batch_size))
#         if not batch:
#             break
#         yield batch
# 
# def get_road_dist_batch(coords):
#     distances = []
#     total_batches = len(coords) // BATCH_SIZE
#     start_time = time.time()
#     for i, coord_batch in enumerate(batch(coords, BATCH_SIZE), 1):
#         batch_start_time = time.time()
#         for start_lon, start_lat, end_lon, end_lat in coord_batch:
#             distance = get_road_dist_cached(start_lon, start_lat, end_lon, end_lat)
#             distances.append(distance)
#         batch_time = time.time() - batch_start_time
#         print(f"Batch {i}/{total_batches} time: {batch_time:.2f} seconds")
#         time.sleep(SLEEP_TIME)  # Sleep to avoid hitting rate limits
#     total_time = time.time() - start_time
#     print(f"Total time for {total_batches} batches: {total_time:.2f} seconds")
#     return distances
# 
# coords = list(df_train[["pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"]].itertuples(index=False, name=None))
# df_train["osrm_dist"] = get_road_dist_batch(coords)

In [ ]:
# import requests
# import pandas as pd
# from concurrent.futures import ThreadPoolExecutor, as_completed
# import time
# import hashlib
# 
# # Initialize a cache
# cache = {}
# 
# def get_road_dist(start_lon, start_lat, end_lon, end_lat):
#     # Create a unique key for the route using hashing for efficiency
#     route_key = hashlib.md5(f"{start_lon},{start_lat},{end_lon},{end_lat}".encode()).hexdigest()
# 
#     # Check if the distance is already in the cache
#     if route_key in cache:
#         return cache[route_key]
# 
#     url = "http://router.project-osrm.org/route/v1/driving/"
#     coords = f"{start_lon},{start_lat};{end_lon},{end_lat}"
#     params = {
#         "geometries": "polyline",
#         "overview": "full",
#         "steps": "true"
#     }
# 
#     try:
#         response = requests.get(url + coords, params=params)
#         response.raise_for_status()
#         data = response.json()
# 
#         if data["code"] == "Ok" and data["routes"]:
#             distance = data["routes"][0]["distance"]
#             # Save the distance in the cache
#             cache[route_key] = distance
#             return distance
#         else:
#             print(f"Error: {data['message']}")
#             return None
#     except requests.exceptions.RequestException as e:
#         print(f"Error: {e}")
#         return None
# 
# def process_row(row):
#     start_time_row = time.time()
#     distance_osrm = get_road_dist(row["pickup_longitude"], row["pickup_latitude"], row["dropoff_longitude"], row["dropoff_latitude"])
#     end_time_row = time.time()
#     print("current row processing time: ", end_time_row - start_time_row)
#     return  distance_osrm
# 
# # Function to handle parallel processing
# def parallel_process(df, func, n_jobs=50):
#     results = []
#     with ThreadPoolExecutor(max_workers=n_jobs) as executor:
#         futures = [executor.submit(func, row) for _, row in df.iterrows()]
#         for future in as_completed(futures):
#             results.append(future.result())
#     return results
# 
# 
# # Apply the function to the dataframe with parallel processing
# start_time = time.time()
# df_train["osrm_dist"] = parallel_process(df_train, process_row, n_jobs=8)
# end_time = time.time()
# 
# print(df_train)
# print(f"Processing time: {end_time - start_time} seconds")

In [ ]:
# import aiohttp
# import asyncio
# import hashlib
# import time
# 
# # Initialize a cache
# cache = {}
# 
# async def get_road_dist(session, start_lon, start_lat, end_lon, end_lat):
#     # Create a unique key for the route using hashing for efficiency
#     route_key = hashlib.md5(f"{start_lon},{start_lat},{end_lon},{end_lat}".encode()).hexdigest()
# 
#     # Check if the distance is already in the cache
#     if route_key in cache:
#         return cache[route_key]
# 
#     url = f"http://router.project-osrm.org/route/v1/driving/{start_lon},{start_lat};{end_lon},{end_lat}"
#     params = {
#         "geometries": "polyline",
#         "overview": "full",
#         "steps": "true"
#     }
# 
#     try:
#         async with session.get(url, params=params) as response:
#             data = await response.json()
#             if data["code"] == "Ok" and data["routes"]:
#                 distance = data["routes"][0]["distance"]
#                 # Save the distance in the cache
#                 cache[route_key] = distance
#                 return distance
#             else:
#                 print(f"Error: {data['message']}")
#                 return None
#     except aiohttp.ClientError as e:
#         print(f"Error: {e}")
#         return None
# 
# async def process_row(session, row):
#     start_time_row = time.time()
#     distance_osrm = await get_road_dist(session, row["pickup_longitude"], row["pickup_latitude"], row["dropoff_longitude"], row["dropoff_latitude"])
#     end_time_row = time.time()
#     print("Current row processing time: ", end_time_row - start_time_row)
#     return distance_osrm
# 
# async def main(df):
#     async with aiohttp.ClientSession() as session:
#         tasks = []
#         for _, row in df.iterrows():
#             tasks.append(process_row(session, row))
#         results = await asyncio.gather(*tasks)
#         return results
# 
# # Apply the function to the dataframe with asynchronous processing
# start_time = time.time()
# df_train["osrm_dist"] = await main(df_train)
# end_time = time.time()
# 
# print(df_train)
# print(f"Total processing time: {end_time - start_time} seconds")